In [1]:
import pandas as pd
from pyspark.sql import SparkSession
from elasticsearch import Elasticsearch, helpers

In [2]:
def salary_check(salary: str):
    if salary is None or salary.strip() == "Cạnh tranh" or salary.strip() == "":
        return ""
    salary = salary.replace("Trên", "").replace("Dưới", "").replace("Tr", "").strip()
    values = salary.split()
    if len(values) == 2:
        if values[1] == "VND":
            return values[0].replace(',', '.')
        if values[1] == "USD":
            return float(values[0].replace(",", "")) * 23182 // 1000000
    if len(values) == 4:
        if values[3] == "VND":
            return f"{values[0].replace(',', '.')} - {values[2].replace(',', '.')}"
        if values[3] == "USD":
            return f"{float(values[0].replace(',', '')) * 23182 // 1000000} - {float(values[2].replace(',', '')) * 23182 // 1000000}"


def degree_check(bc):
    if bc is None or bc == "" or bc.strip() == "Khác":
        return "Không yêu cầu"
    return bc.strip()


def experience_check(kn: str):
    if kn is None or kn == "" or kn.strip() == "Chưa có kinh nghiệm" or kn.strip() == "0 - 0 Năm":
        return "Không yêu cầu"
    kns = kn.replace("Năm", "").strip().split()
    if len(kns) == 2:
        return kns[0]
    if kns[0] >= kns[2]:
        return kns[0]
    else:
        return f"{kns[0]} - {kns[2]}"


def sex_check(sex: str):
    if sex is None or sex == "":
        return "Không yêu cầu"
    return sex.strip()

In [3]:
spark = SparkSession.builder.config("spark.sql.debug.maxToStringFields", 100000).getOrCreate()
df = spark.read.json("careerbuilder-raw.json").toPandas()
df["update_time"] = pd.to_datetime(df["update_time"], format="%d/%m/%Y").dt.strftime("%Y-%m-%d")
df["sex"] = df["sex"].apply(sex_check)
df = df[df["company_name"].str.strip() != ""]
df["salary"] = df["salary"].apply(salary_check)
df["degree"] = df["degree"].apply(degree_check)
df["experience"] = df["experience"].apply(experience_check)
df = spark.createDataFrame(df.astype(str)).toPandas()

22/07/05 17:28:22 WARN Utils: Your hostname, ubuntu resolves to a loopback address: 127.0.1.1; using 192.168.1.133 instead (on interface enp6s0)
22/07/05 17:28:22 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/07/05 17:28:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [8]:
es = Elasticsearch("http://172.18.0.25:9200")


def generator():
    documents = []
    for i in range(0, 5926):
        document = {
            "id_": i,
            "age": df.iloc[i: i + 1].age, 
            "company_name": df.iloc[i: i + 1].company_name, 
            "degree": df.iloc[i: i + 1].degree, 
            "experience": df.iloc[i: i + 1].experience, 
            "job_field": df.iloc[i: i + 1].job_field, 
            "job_name": df.iloc[i: i + 1].job_name, 
            "level": df.iloc[i: i + 1].level, 
            "location": df.iloc[i: i + 1].location, 
            "salary": df.iloc[i: i + 1].salary, 
            "sex": df.iloc[i: i + 1].sex, 
            "update_time": df.iloc[i: i + 1].update_time, 
            "working_form": df.iloc[i: i + 1].working_form, 
        }
        documents.append(document)
    helpers.bulk(es, documents, index="job-tuoitre365")

In [9]:
generator()

In [2]:
es = Elasticsearch("http://172.18.0.25:9200")

In [14]:
es.sql.query(body={'query': 'SELECT count(DISTINCT location) FROM "job-*"'})['rows'][0][0]

/tmp/ipykernel_230713/3818646108.py:1: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es.sql.query(body={'query': 'SELECT count(DISTINCT location) FROM "job-*"'})['rows'][0][0]


69

In [49]:
import pandas as pd
import plotly.express as px

In [118]:
es.sql.query(body={'query': 'SELECT * FROM "job-*" WHERE job_field LIKE \'%Tư vấn%\' LIMIT 2'})['rows']

/tmp/ipykernel_230713/3623477235.py:1: DeprecationWarning:

The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.



[[' Không giới hạn tuổi',
  'CÔNG TY CỔ PHẦN VUA NỆM',
  'Cao đẳng',
  '2 - 3',
  0,
  "['Bán lẻ / Bán sỉ', 'Bán hàng / Kinh doanh', 'Tư vấn']",
  'Cửa Hàng Trưởng Biên Hòa',
  'Quản lý',
  'Đồng Nai',
  '8 - 15',
  'Không yêu cầu',
  '2021-12-19T00:00:00.000Z',
  'Nhân viên chính thức'],
 [' Không giới hạn tuổi',
  'CÔNG TY CỔ PHẦN VUA NỆM',
  'Cao đẳng',
  '2 - 3',
  0,
  "['Bán lẻ / Bán sỉ', 'Bán hàng / Kinh doanh', 'Tư vấn']",
  'Cửa Hàng Trưởng Biên Hòa',
  'Quản lý',
  'Đồng Nai',
  '8 - 15',
  'Không yêu cầu',
  '2021-12-19T00:00:00.000Z',
  'Nhân viên chính thức']]

In [107]:
df = pd.DataFrame(results, columns=["location", "number of records"])

In [108]:
figure = px.bar(df, x="location", y="number of records");

In [109]:
figure.show()

In [119]:
es.sql.query(body={'query': f'SELECT DISTINCT (age, company_name, degree, experience, job_field, job_name, level, location, salary, sex, update_time, working_form) FROM "job-*" WHERE company_name LIKE \'%{company_name}%\' AND location LIKE \'%{location}%\' AND job_name LIKE \'%{job_name}%\' AND job_field LIKE \'%{job_field}%\' AND degree LIKE \'%{degree}%\' LIMIT 10'})['rows']

NameError: name 'company_name' is not defined

In [120]:
df.to_json().encode("utf8")

b'{"location":{"0":"H\\u1ed3 Ch\\u00ed Minh","1":"H\\u00e0 N\\u1ed9i","2":"B\\u00ecnh D\\u01b0\\u01a1ng","3":"\\u0110\\u1ed3ng Nai","4":"Long An","5":"B\\u00e0 R\\u1ecba - V\\u0169ng T\\u00e0u","6":"\\u0110\\u00e0 N\\u1eb5ng","7":"H\\u1ea3i Ph\\u00f2ng","8":"H\\u01b0ng Y\\u00ean","9":"H\\u00e0 Nam"},"number of records":{"0":7287,"1":5847,"2":882,"3":636,"4":246,"5":216,"6":210,"7":159,"8":141,"9":117}}'